In [ ]:
!pip install -q nlplot
!pip install -q python-Levenshtein

In [ ]:
import pandas as pd
import nlplot
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import Levenshtein
df = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
titles = pd.read_csv("../input/cpc-codes/titles.csv")
df = pd.merge(df,titles,left_on="context",right_on="code")

# コンペ概要
あなたは、発明から得られた大規模なテキストベースのデータセットから意味を抽出することができますか？今がそのチャンスです。

米国特許商標庁（USPTO）は、そのオープンデータポータルを通じて、科学、技術、商業に関する世界最大級の情報リポジトリを提供しています。特許は、新規かつ有用な発明の公開と引き換えに与えられる知的財産の一形態である。特許は付与される前に徹底的な審査が行われ、米国の技術革新の歴史は2世紀以上、1100万件の特許に及ぶため、米国特許アーカイブはデータ量、質、多様性の点で稀有な存在となっています。

「USPTOは、特許を付与し、商標を登録し、世界中で知的財産を促進することで、眠らないアメリカのイノベーションマシンに貢献しています。USPTOは、電球から量子コンピュータに至るまで、200年以上にわたる人類の創意工夫を世界と共有しています。データサイエンス・コミュニティの創造性と組み合わせることで、USPTOのデータセットは、科学と社会全体の進歩に貢献するAIとMLモデルを強化する無限の可能性を持っています。

USPTO最高情報責任者 Jamie Holcombe

このコンペティションでは、新しい意味的類似性データセットでモデルを学習させ、特許文書内のキーフレーズをマッチングさせて関連情報を抽出することになります。フレーズ間の意味的な類似性を判断することは、特許検索や審査プロセスにおいて、ある発明が以前に記載されていたかどうかを判断するために非常に重要です。例えば、ある発明が「テレビ」を主張し、先行文献に「テレビセット」が記載されている場合、モデルは理想的にはこれらが同じであると認識し、弁理士や審査官が関連文書を検索するのを支援することができます。これは言い換えの識別にとどまらず、ある発明が「強い材料」と主張し、別の発明が「鋼鉄」を使用している場合にも一致する可能性があります。ある発明が「強い材料」を主張し、別の発明が「鋼鉄」を使用している場合、これも一致する場合があります。「強い材料」の定義は領域によって異なります（ある領域では鋼鉄、別の領域ではリップストップ生地かもしれませんが、パラシュートが鋼鉄製であることは好ましくないでしょう）。このような状況を曖昧にしないための追加機能として、技術的なドメインコンテキストとして特許協力分類を搭載しています。

文脈情報を抽出するためにフレーズをマッチングするモデルを構築し、それによって、特許コミュニティが何百万もの特許文書間の点をつなげるのを助けることができますか？


# データ内容

|  カラム名  |  内容  | 
| ---- | ---- |
|  id  |   phraseのペアを表す一意の識別子  |
|  anchor  |  1番目のphrase  |
|  target  |  2番目のphrase  |
|  context  |  CPC分類（バージョン2021.05）、類似度が得点化される対象を示す |
|  score  |  類似性(目的変数)  |

# 目的変数Scoreについて
スコアは0～1の範囲で、0.25刻みで以下の意味を持つ。

- 1.0 - 極めて近い一致。これは、活用、数量（例：単数または複数）、ストップワード（例：「the」、「and」、「or」）の追加または削除の違いを除いて、完全に一致すること。<br>
- 0.75 - 近い同義語、例：「mobile phone」と「cellphone」。また、「TCP」→「transmission control protocol」のような略語も含まれる。<br>
- 0.5 - 同じ意味（同じ機能、同じ性質）を持たない同義語。これには、広義-狭義（※下位概念）および狭義-広義（※上位概念）一致が含まれます。<br>
- 0.25 - やや関連性がある。例えば、2つのフレーズは同じ高レベルドメインにあるが、同義語ではない。これには反意語も含まれます。<br>
- 0.0 - 関連性がない。<br><br>

※上位概念,下位概念<br>
ある言葉に対して、抽象化した言葉を上位概念、具体化した言葉を下位概念という。<br>
ある単語の上位概念及び下位概念は一つとは限らない。<br>
(一例)野菜にとって食べ物は上位概念で、トマトは下位概念。<br>


# anchorとtarget
| 変数名 |  関係  |  平均score  | 行数|
| ---- | ---- | ---- | ---- |
| allsame |  anchorとtargetが完全一致  | 1.0 | 279 |
| acont | anchorの中にtargetが含まれている  |  0.4933906146728354  | 1513 |
| tcona | targetの中にanchorが含まれている  |  0.5720884658454647  | 1786 |
| others |  その他  | 0.33920808633530931 | 32895 |
| df |  全部  | 0.36206234748992405 | 36473 |

※acont,tconaの中にallsameのものは含まない

In [ ]:
def target_contains_anchor(row):
    return row.anchor in row.target
def anchor_contains_target(row):
    return row.target in row.anchor
def L_dist_target_and_anchor(row):
    return Levenshtein.distance(row.anchor,row.target)

In [ ]:
df["L_dist"] = df.apply(L_dist_target_and_anchor,axis=1)
allsame = df[df["anchor"]==df["target"]]
tcona = df[df.apply(target_contains_anchor,axis=1)&(df["anchor"]!=df["target"])]
acont = df[df.apply(anchor_contains_target,axis=1)&(df["anchor"]!=df["target"])]
others = df[~(df.apply(target_contains_anchor,axis=1)+df.apply(anchor_contains_target,axis=1))&(df["anchor"]!=df["target"])]

In [ ]:
tcona.head(3)

In [ ]:
acont.head(3)

In [ ]:
def plot_bar_ngram(df,target_col):
    npt = nlplot.NLPlot(df,target_col=target_col)
    stopwords = npt.get_stopword(top_n=0, min_freq=0)
    figure = npt.bar_ngram(
             title='uni-gram',
             xaxis_label='word_count',
             yaxis_label='word',
             ngram=1,
             top_n=50,
    )
    return figure

In [ ]:
def display_ngram(target_col):
    fig1 =plot_bar_ngram(allsame,target_col)
    fig2 = plot_bar_ngram(tcona,target_col)
    fig3 =plot_bar_ngram(acont,target_col)
    fig4 = plot_bar_ngram(others,target_col)
    trace1 = fig1['data'][0]
    trace2 = fig2['data'][0]
    trace3 = fig3['data'][0]
    trace4 = fig4['data'][0]

    fig = make_subplots(rows=2, cols=2, 
                        subplot_titles=('allsame', 'tcona','acont',"others"), shared_xaxes=False,
                       vertical_spacing=0.05)
    fig.update_xaxes(title_text='word count', row=1, col=1)
    fig.update_xaxes(title_text='word count', row=1, col=2)
    fig.update_xaxes(title_text='word count', row=2, col=1)
    fig.update_xaxes(title_text='word count', row=2, col=2)

    fig.update_layout(height=2000, width=1000,title_text="unigram "+target_col)
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)
    fig.add_trace(trace3, row=2, col=1)
    fig.add_trace(trace4, row=2, col=2)

    fig.show()
    
def comp_anchor_target():
    fig1 = plot_bar_ngram(tcona,"anchor")
    fig2 = plot_bar_ngram(tcona,"target")
    fig3 =plot_bar_ngram(acont,"anchor")
    fig4 =plot_bar_ngram(acont,"target")
    fig5 = plot_bar_ngram(others,"anchor")
    fig6 = plot_bar_ngram(others,"target")
    trace1 = fig1['data'][0]
    trace2 = fig2['data'][0]
    trace3 = fig3['data'][0]
    trace4 = fig4['data'][0]
    trace5 = fig5['data'][0]
    trace6 = fig6['data'][0]

    fig = make_subplots(rows=3, cols=2, 
                        subplot_titles=('tcona anchor','tcona target',
                                        'acont anchor','acont target',
                                        "others anchor",'othres target'), shared_xaxes=False,
                       vertical_spacing=0.05)
    fig.update_xaxes(title_text='word count', row=1, col=1)
    fig.update_xaxes(title_text='word count', row=1, col=2)
    fig.update_xaxes(title_text='word count', row=2, col=1)
    fig.update_xaxes(title_text='word count', row=2, col=2)
    fig.update_xaxes(title_text='word count', row=3, col=2)
    fig.update_xaxes(title_text='word count', row=3, col=2)

    fig.update_layout(height=3000, width=1000,title_text="anchor vs target")
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)
    fig.add_trace(trace3, row=2, col=1)
    fig.add_trace(trace4, row=2, col=2)
    fig.add_trace(trace5, row=3, col=1)
    fig.add_trace(trace6, row=3, col=2)

    fig.show()

In [ ]:
display_ngram("anchor")

In [ ]:
display_ngram("target")

In [ ]:
comp_anchor_target()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["score"].astype(str).sort_values(),histnorm="probability",name="df"))
fig.update_layout(title="Score Probability")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["score"].astype(str).sort_values(),histnorm="probability",name="df"))
fig.add_trace(go.Histogram(x=tcona["score"].astype(str),histnorm="probability",name="tcona"))
fig.add_trace(go.Histogram(x=acont["score"].astype(str),histnorm="probability",name="acont"))
fig.add_trace(go.Histogram(x=others["score"].astype(str),histnorm="probability",name="others"))
fig.update_layout(title="Score Probability")

# レーベンシュタイン距離
二つの文字列がどの程度異なっているかを示す距離。<br>
https://ja.wikipedia.org/wiki/%E3%83%AC%E3%83%BC%E3%83%99%E3%83%B3%E3%82%B7%E3%83%A5%E3%82%BF%E3%82%A4%E3%83%B3%E8%B7%9D%E9%9B%A2

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["L_dist"].sort_values().astype(str),histnorm="probability",name="df"))
fig.update_layout(title="Levenshtein_distance_between_anchor_and_target")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df["score"]==1]["L_dist"].sort_values()   ,name="score 1"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.75]["L_dist"].sort_values(),name="score 0.75"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.5]["L_dist"].sort_values() ,name="score 0.5"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.25]["L_dist"].sort_values(),name="score 0.25"))
fig.add_trace(go.Histogram(x=df[df["score"]==0]["L_dist"].sort_values()   ,name="score 0"))
fig.update_layout(title="Levenshtein_distance_between_anchor_and_target",
                 xaxis=dict(title="Levenshtein_distance"),
                 yaxis=dict(title="score_count"))

In [ ]:
df.corr()

# 共通する単語
レーベンシュタイン距離は語順を考慮していないので、reading book と　book reading だと距離が非常に遠くなる。<br>
単純に共通する単語をカウントしてみる。

In [ ]:
def len_common_words(row):
    anchor_words = row.anchor.split()
    target_words = row.target.split()
    return len(set(anchor_words)&set(target_words))

In [ ]:
df["common_words"] = df.apply(len_common_words,axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["common_words"].sort_values().astype(str),histnorm="probability",name="df"))
fig.update_layout(title="common_words")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df["score"]==1]["common_words"].sort_values()   ,name="score 1"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.75]["common_words"].sort_values(),name="score 0.75"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.5]["common_words"].sort_values() ,name="score 0.5"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.25]["common_words"].sort_values(),name="score 0.25"))
fig.add_trace(go.Histogram(x=df[df["score"]==0]["common_words"].sort_values()   ,name="score 0"))
fig.update_layout(title="Levenshtein_distance_between_anchor_and_target",
                 xaxis=dict(title="common_word_num"),
                 yaxis=dict(title="score_count"))

# 略語
National Aeronautics and Space Administration　⇒ NASA<br>
のように略語の関係の場合はscoreは0.75になると書いてあるので、前処理で作っておいてよいかもしれない

In [ ]:
def make_abbreviation(words):
    word_list = words.split()
    abbreviation = ""
    for word in word_list:
        abbreviation+=word[0]
    return  abbreviation

In [ ]:
df["anchor_abb"] = df["anchor"].apply(make_abbreviation)
df["target_abb"] = df["target"].apply(make_abbreviation)

In [ ]:
df[df["anchor_abb"]==df["target"]]

In [ ]:
df[df["target_abb"]==df["anchor"]]

In [ ]:
def L_dist_target_and_anchor_abb(row):
    return Levenshtein.distance(row.anchor_abb,row.target)
def L_dist_target_abb_and_anchor(row):
    return Levenshtein.distance(row.anchor,row.target_abb)

In [ ]:
df["L_dist_2"] = df.apply(L_dist_target_and_anchor_abb,axis=1)
df["L_dist_3"] = df.apply(L_dist_target_abb_and_anchor,axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df["score"]==1]["L_dist_2"].sort_values()   ,name="score 1"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.75]["L_dist_2"].sort_values(),name="score 0.75"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.5]["L_dist_2"].sort_values() ,name="score 0.5"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.25]["L_dist_2"].sort_values(),name="score 0.25"))
fig.add_trace(go.Histogram(x=df[df["score"]==0]["L_dist_2"].sort_values()   ,name="score 0"))
fig.update_layout(title="Levenshtein_distance_between_anchor_abb_and_target",
                 xaxis=dict(title="Levenshtein_distance"),
                 yaxis=dict(title="score"))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df["score"]==1]["L_dist_3"].sort_values()   ,name="score 1"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.75]["L_dist_3"].sort_values(),name="score 0.75"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.5]["L_dist_3"].sort_values() ,name="score 0.5"))
fig.add_trace(go.Histogram(x=df[df["score"]==0.25]["L_dist_3"].sort_values(),name="score 0.25"))
fig.add_trace(go.Histogram(x=df[df["score"]==0]["L_dist_3"].sort_values()   ,name="score 0"))
fig.update_layout(title="Levenshtein_distance_between_anchor_and_target_abb",
                 xaxis=dict(title="Levenshtein_distance"),
                 yaxis=dict(title="score"))

# 所感
anchor target間の内包:0,0.25が明らかに少ないので精度に貢献できそう？<br>
レーベンシュタイン距離：距離が短いものに関しては精度に貢献できそう？<br>
共通する単語：数によって明確にscoreの分布が代わるが、anchor target間の内包とやや被っている？<br>
略語：意図的かtargetにいくつかの略語の候補が複数列挙されているので効果は薄そう<br>